- decesion tree with max depth of 3

In [1]:
import pandas as pd
import os
import random
from datetime import timedelta
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import numpy as np

In [444]:
files = os.listdir('data_output/')
files = [i for i in files if 'csv' in i]
#random.sample(files,10)

In [445]:
# empty list for confusion scores
output = []

# set up the csv for actuals data
header = pd.DataFrame(columns=['date_time','mean_count', 'y', 'y_pred', 'station_id', 'split'])
header.to_csv('data_decisiontree/deciscion_tree_predictions.csv', mode='a', index=False, header=True)

In [446]:
# load the station to model
station_id = files[1]
url = 'data_output/'
data = pd.read_csv(url+str(station_id), index_col='date_time', parse_dates=True)

In [447]:
# self join data set offset on hours
time_offset = 4
feature = pd.read_csv(url+str(station_id), index_col='date_time', parse_dates=True)
feature.index = feature.index + timedelta(minutes=(time_offset*30))
columns = [str(4*30)+'_'+ name for name in feature.columns]
feature.columns = columns
data_feature = pd.concat([data,feature], axis=1, join='inner')

In [448]:
# y class
data_feature['minutes_empty'] = (data_feature.minutes_empty > 0).astype(int)

# X and y
feature_cols = [ i for i in data_feature.columns[2:] if 'full' not in i and 'empty' not in i and str(time_offset*30) in i]
X = data_feature[feature_cols]
y = data_feature[['minutes_empty']]

In [449]:
# only run the model if it meets a .5% threshold of events
if (y.minutes_empty > 0).sum() / (len(y) * 1.0) < .01:
    print 'nothing'
else:
    print station_id + 'model'

101.csvmodel


In [450]:
# train test split (75% train, 25% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# I wante to do this as a time series, but the ends of the data sets are genrally skewed

#X_train = X.head(int(len(X) * .5))
#y_train = y.head(int(len(X) * .5))
#X_test = X.tail(len(X) - int(len(X) * .5))
#y_test = y.tail(len(X) - int(len(X) * .5))

In [451]:
# tree classification fit
treeclf = DecisionTreeClassifier(max_depth = 3, random_state=1)
treeclf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1, splitter='best')

In [439]:
# predictions
y_pred = treeclf.predict(X_test)
confusion = metrics.confusion_matrix(y_test, y_pred)

# columns = ['TN','FN', 'FP', 'TP']
confusion = confusion[0].tolist() + confusion[1].tolist()

# add rows to final
row = [int(station_id[:station_id.find('.csv')])] + confusion
output.append(row)

In [456]:
y_pred = treeclf.predict(X_test)
confusion = metrics.confusion_matrix(y_test, y_pred)
confusion[0].tolist() + confusion[1].tolist()

[5159, 1, 1, 61]

In [440]:
# y actual, y predicted, mean count actuals
y_pred_all = treeclf.predict(X)
y_pred_df = pd.DataFrame(y_pred_all, index=y.index, columns=['y_pred'])
y.columns = ['y']
mean_count = data[['mean_count']]

# output df
output_actuals = pd.concat([mean_count, y, y_pred_df], axis=1)
output_actuals['station_id'] = int(station_id[:station_id.find('.csv')])

# label as train or test
y_train['split'] = 'train'
y_test['split'] = 'test'
test_train = pd.concat([y_train, y_test], axis=0).sort_index()
output_actuals = pd.concat( [output_actuals, test_train[['split']]], axis=1 )

output_actuals.to_csv('data_decisiontree/deciscion_tree_predictions.csv',index=True, mode='a', header=False)

/Users/bshonk/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bshonk/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [367]:
# convert to DF and output to csv
output = pd.DataFrame(output, columns=['station_id', 'TN','FN', 'FP', 'TP'])
output.to_csv('data_decisiontree/deciscion_tree.csv',index=False)